In [1]:
import numpy as np
import pandas as pd
import openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import plotly.express as px
from plotly.subplots import make_subplots
import dimod
from dwave.system import LeapHybridCQMSampler
from dwave.plugins.sklearn import SelectFromQuadraticModel

dataset = openml.datasets.get_dataset(312)

X, y, categorical_indicator, attribute_names = dataset.get_data(
    target=dataset.default_target_attribute, dataset_format="dataframe"
)
X = X.astype(float)
y = y.values.astype(int)

clf = RandomForestClassifier()

dataset


OpenML Dataset
Name..........: scene
Version.......: 1
Format........: ARFF
Upload Date...: 2014-08-25 11:43:22
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/1390080/scene.arff
OpenML URL....: https://www.openml.org/d/312
# of features.: 300
# of instances: 2407

In [2]:
dataset.get_data()[0]


,attr1,attr2,attr3,attr4,attr5,attr6,attr7,attr8,attr9,attr10,...,attr291,attr292,attr293,attr294,Beach,Sunset,FallFoliage,Field,Mountain,Urban
0,0.646467,0.666435,0.685047,0.699053,0.652746,0.407864,0.150309,0.535193,0.555689,0.580782,...,0.157332,0.247298,0.014025,0.029709,1,0,0,0,1,0
1,0.770156,0.767255,0.761053,0.745630,0.742231,0.688086,0.708416,0.757351,0.760633,0.740314,...,0.251454,0.137833,0.082672,0.036320,1,0,0,0,0,1
2,0.793984,0.772096,0.761820,0.762213,0.740569,0.734361,0.722677,0.849128,0.839607,0.812746,...,0.017166,0.051125,0.112506,0.083924,1,0,0,0,0,0
3,0.938563,0.949260,0.955621,0.966743,0.968649,0.869619,0.696925,0.953460,0.959631,0.966320,...,0.019267,0.031290,0.049780,0.090959,1,0,0,0,0,0
4,0.512130,0.524684,0.520020,0.504467,0.471209,0.417654,0.364292,0.562266,0.588592,0.584449,...,0.198151,0.238796,0.164270,0.184290,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402,0.875782,0.901653,0.926227,0.721366,0.795826,0.867642,0.794125,0.899067,0.908963,0.895336,...,0.215147,0.279607,0.254413,0.134350,0,0,0,0,0,1
2403,0.657706,0.669877,0.692338,0.713920,0.727374,0.750354,0.684372,0.718770,0.719916,0.730645,...,0.217201,0.199491,0.048747,0.041638,0,0,0,0,0,1
2404,0.952281,0.944987,0.905556,0.836604,0.875916,0.957034,0.953938,0.967956,0.819636,0.707311,...,0.028002,0.031900,0.017547,0.019734,0,0,0,0,0,1
2405,0.883990,0.899004,0.901019,0.904298,0.846402,0.858145,0.851362,0.852472,0.876665,0.908187,...,0.239041,0.256158,0.226332,0.223070,0,0,0,0,0,1


In [9]:
def evaluate_model(m, X, y, indices=None):
    if not indices is None:
        X_filtered = X.iloc[:, indices]
    else:
        X_filtered = X
    acc = np.mean(cross_val_score(clf, X_filtered, y, cv=5))
    return acc


def show_relevance_redundancy(X, y, indices=None, title=""):
    if not indices is None:
        X = X.iloc[:, indices].copy()
    y = y
    fig = make_subplots(
        rows=1,
        cols=2,
        column_widths=[0.68, 0.32],
        column_titles=["relevance", "redundancy"],
    )
    trace_rel = px.bar(np.array([abs(np.corrcoef(x, y)[0, 1]) for x in X.values.T]))
    trace_red = px.imshow(abs(np.corrcoef(X.values, rowvar=False)))
    fig.add_trace(trace_rel.data[0], row=1, col=1)
    fig.add_trace(trace_red.data[0], row=1, col=2)
    fig.update_layout(width=1200, height=480, title=title)
    fig.show()


show_relevance_redundancy(
    X,
    y,
    None,
    f"full dataset: acc={evaluate_model(clf, X, y, None):.4f}",
)


$$\beta \sum LinearTerms + (1 - \beta) / (2 (k - 1)) \sum QuadTerms$$

$$\alpha \sum LinearTerms - (1 - \alpha) \sum QuadTerms$$

$$-(\alpha \sum_i x_i c_{ii} - (1 - \alpha) \sum_i \sum_{j \neq i} x_i x_j c_{ij})$$

In [4]:
beta = 0.5
k = 30
# Pearson correlation
correlation_matrix = abs(np.corrcoef(np.hstack((X, y[:, np.newaxis])), rowvar=False))
# fix the alpha parameter from the Milne paper
# to account for the numerous quadratic terms that are possible
alpha = 2 * beta * (k - 1) / (1 - beta + 2 * beta * (k - 1))
# generate weights for linear and quadratic terms, per Milne algorithm
Rxy = correlation_matrix[:, -1]
Q = correlation_matrix[:-1, :-1] * (1 - alpha)
np.fill_diagonal(Q, -Rxy * alpha)
# create binary quadratic model from the linear and quadratic weights
bqm = dimod.BinaryQuadraticModel(Q, "BINARY")
# create constrained quadratic model
cqm = dimod.ConstrainedQuadraticModel()
# the objective function of the CQM is the same as BQM
cqm.set_objective(bqm)
# constraint: limit the number of features to k
cqm.add_constraint_from_iterable(
    ((i, 1) for i in range(len(cqm.variables))), "==", rhs=k
)
# the sampler that will be used is the hybrid sampler in the DWave cloud
sampler = LeapHybridCQMSampler()
# sampler_args.setdefault("label", "Feature Selection")
# solve the problem
# sampleset = sampler.sample_cqm(cqm, **sampler_args)
sampleset = sampler.sample_cqm(cqm)

# postprocess results
feasible = sampleset.filter(lambda s: s.is_feasible)
if feasible:
    best = feasible.first
else:
    assert len(cqm.constraints) == 1
    best = sorted(
        sampleset.data(),
        key=lambda x: (list(cqm.violations(x.sample).values())[0], x.energy),
    )[0]
assert list(best.sample.keys()) == sorted(best.sample.keys())
is_selected = np.array([bool(val) for val in best.sample.values()])
features = np.array([i for i, val in enumerate(is_selected) if val])
best_energy = best.energy


In [5]:
print(features)
print(best_energy)


[ 14  20  79  82  84  87  88  89  90  91  93  95  96 102 105 171 208 215
 225 238 239 240 243 244 268 269 294 295 296 298]
-4.3434692798078105


In [6]:
show_relevance_redundancy(
    X,
    y,
    features,
    f"explicit optimization: acc={evaluate_model(clf, X, y, features):.4f}",
)


In [7]:
X_new = SelectFromQuadraticModel(num_features=30, alpha=0.5).fit_transform(X, y)
X_new_df = pd.DataFrame(data=X_new, columns=list(range(X_new.shape[1])))


C:\Users\flori\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:432: UserWarning:

X has feature names, but SelectFromQuadraticModel was fitted without feature names



In [10]:
show_relevance_redundancy(
    X_new_df,
    y,
    None,
    f"plugin optimization: acc={evaluate_model(clf, X_new_df, y, None):.4f}",
)
